In [1]:
import mhcflurry
from mhcflurry import Class1PresentationPredictor

%matplotlib inline

import re
import os
import sys
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Align
import matplotlib.pyplot as plt
import pandas as pd
from joblib import load
from igraph import Graph as igraph_graph, plot, rescale

# adding project folder to lib search path
project = "/".join((os.path.split(os.getcwd())[0], os.path.split(os.getcwd())[1], 'scripts'))
if project not in sys.path:
    sys.path.append(project)
    
import loader
import work_with_files
import classificator_k_mer
import tree_building
import patients_data
import data_prep_k_mer
import quantitative_analyze
import calculating_metrics

### Looking at p1

In [2]:
# loading pat

pat = patients_data.Patient('p1')

pat_reg_data = pat.regions.region.loc[pat.regions.region.name == 'V3']

In [3]:
# getting only peptides

pat_peptides = list(pat_reg_data.translated.apply(str))

In [4]:
# cutting peptides into pieces

pat_pept_pieces = []

for pepti in pat_peptides:
    i = 9
    j = 0
    
    length = len(pepti)
    
    while i < length:
        
        # will drop pieces with stop codon
        if '*' in pepti[j: i]:
            print(pepti[j: i])
            i += 2
            j += 2
            continue
        
        pat_pept_pieces.append(pepti[j: i])
        i += 2
        j += 2

NIHNSAP*R
HNSAP*RTC
SAP*RTCRN
P*RTCRNQL
RTRTSIL*N
RTSIL*NRR
SIL*NRRHN
L*NRRHNRR
KKSILCN*W
SILCN*WNK
LCN*WNKME
N*WNKMERN
ERNFKTGS*
NFKTGS*KI
KTGS*KIKK
GS*KIKKVL
*KIKKVL**
IKKVL**DN
KVL**DNSL
L**DNSL*T
*DNSL*TTL
NSL*TTLRR
L*TTLRRRS
EKLKKVL**
LKKVL**DN
KVL**DNSL
L**DNSL*T
*DNSL*TTL
NSL*TTLRR
L*TTLRRRS


In [5]:
# settings alleles

pat_hla = ['HLA-A*02:02', 'HLA-B*08:15']

In [6]:
# making predictior and making him doing his stuff

predictor = Class1PresentationPredictor.load()

pept_pred = predictor.predict(peptides=pat_pept_pieces,
                  alleles=pat_hla,
                  verbose=0)

Using TensorFlow backend.


In [9]:
pept_pred.shape

(5632, 7)

In [8]:
# watching affinity

pept_pred.loc[pept_pred.affinity < 500].shape

(441, 7)